In [1]:
import pandas as pd
from Bio import SeqIO

### Import RNA-seq data (TPM) for alpha and gamma-gliadin genes.

In [54]:
data_RNA = pd.read_csv("./Input/RNA-seq_alpha_gamma_BW208.txt", sep = "\t", header = 0, index_col = 0)
data_RNA

,Protein.type,BW208,BW208.1,BW208.2
Gene,,,,
TraesCS6A02G048900,Alpha gliadin,0.264840,0.109270,0.000000
TraesCS6A02G049066,Alpha gliadin,0.652935,0.026523,0.023725
TraesCS6A02G049100,Alpha gliadin,527.868000,62.876000,39.433700
TraesCS6A02G049200,Alpha gliadin,385.137000,48.879800,23.420600
TraesCS6A02G049400,Alpha gliadin,3.265410,0.904653,0.312157
...,...,...,...,...
TraesCS1D02G001000,Gamma gliadin,558.490000,56.040200,36.976600
TraesCS1D02G001100,Gamma gliadin,1285.980000,172.933000,98.335000
TraesCS1D02G001200,Gamma gliadin,4440.130000,503.041000,405.251000


### Import epitope sequences.

In [26]:
epitope_dict = {}

epitopeFile = "./Input/epitopes.fasta"
for record in SeqIO.parse(epitopeFile, "fasta"):
    epitope_dict[str(record.id)] = str(record.seq)

In [27]:
peptide_dict = {}

peptideFile = "./Input/genome_alpha_gamma.fasta"
for record in SeqIO.parse(peptideFile, "fasta"):
    peptide_dict[str(record.id)] = str(record.seq)

In [31]:
def occurrences(string, sub):
    count = start = 0
    while True:
        start = string.find(sub, start) + 1
        if start > 0:
            count+=1
        else:
            return count

In [34]:
oc_dict = {}
for peptide_id, peptide_seq in peptide_dict.items():
    oc_list = []
    for epitope_id, epitope_seq in epitope_dict.items():
        if epitope_seq in peptide_seq:
            oc = occurrences(peptide_seq, epitope_seq)
        else:
            oc = 0
        oc_list.append(oc)
    oc_dict[peptide_id.split("|")[0]] = oc_list

In [57]:
data_oc = pd.DataFrame(oc_dict).transpose()
data_oc.columns = epitope_dict.keys()
data_final = pd.concat([data_RNA, data_oc], axis = 1)

In [73]:
# alpha:
samples = ["BW208", "BW208.1", "BW208.2"]
data_alpha_RNA = data_final[data_final["Protein.type"] == "Alpha gliadin"]

alpha_dict = {}

for epitope_id in epitope_dict.keys():
    for sample in samples:
        alpha_dict.setdefault(epitope_id, []).append(sum(data_alpha_RNA[sample]*data_alpha_RNA[epitope_id]))
data_alpha = pd.DataFrame(alpha_dict).transpose()
data_alpha.columns = [i + "_alpha" for i in samples]

In [78]:
# gamma:
samples = ["BW208", "BW208.1", "BW208.2"]
data_gamma_RNA = data_final[data_final["Protein.type"] == "Gamma gliadin"]

gamma_dict = {}

for epitope_id in epitope_dict.keys():
    for sample in samples:
        gamma_dict.setdefault(epitope_id, []).append(sum(data_gamma_RNA[sample]*data_gamma_RNA[epitope_id]))
data_gamma = pd.DataFrame(gamma_dict).transpose()
data_gamma.columns = [i + "_gamma" for i in samples]

In [81]:
data_alpha_gamma = pd.concat([data_alpha, data_gamma], axis = 1)
data_alpha_gamma.to_csv("./results/RNA_epitopes.txt", sep = "\t")